# Agent for Manual Automotive Query with RAG

## Import Libraries

In [58]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores.milvus import Milvus
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from PyPDF2 import PdfReader
import os
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from pymilvus import connections, Collection, utility, MilvusClient
import shutil
from langchain.tools import tool

## Configuration of the variables

In [59]:
OPENAI_API_KEY = "Your OpenAI API Key"
MILVUS_HOST = 'Your Milvus Host'
MILVUS_PORT = 'Your Milvus Port'
COLLECTION_NAME = 'Your Collection Name'

## Define the brand, model and year of the car, and the query

In [179]:
# brand = "Volkswagen"
# model = "Gol"
# year = "2022"

# Embeddings Model
embeddings = OpenAIEmbeddings(chunk_size=500, openai_api_key=OPENAI_API_KEY, model='text-embedding-3-small')

brand = "Volkswagen"
model = "Taos"
year = "2023"
#01 -----------------------------------------------------------------------
# problem_description = "When will the alarm be triggered?"

#02 -----------------------------------------------------------------------
# problem_description = "When is the air recirculation mode turned off?"

#03 -----------------------------------------------------------------------
problem_description = "When should the seat heating not be turned on?"

template_system = f"""
                  You are a customer service agent at a car dealership who must assist customers with their issues. The customer's car is a {brand} {model} from {year}. First, check the availability of the {brand} {model} {year} car in the knowledge base. Inform the customer whether the model is available or not. Then, research in the knowledge base how to resolve the '{problem_description}' problem for the {brand} {model} {year} and provide a detailed response to the customer with the steps for the solution, if the model is available. If the model is not available, inform the customer that you do not have detailed information about the {brand} {model} {year} in your system.

                  Example 1:
                    Agent's input - 'Please check the availability of the Toyota Corolla 2020 car and inform the customer if the model is available. Then, research in the knowledge base how to resolve the overheating problem for the Toyota Corolla 2020 and provide a detailed response to the customer with the steps for the solution.'
                    Expected response - 'Hello! I've checked that you have a Toyota Corolla 2020. I understand you're facing an overheating issue. I'll assist you with that. Please check if the coolant level is adequate and if the radiator hoses are not damaged or obstructed. If you need further assistance or have other questions, I'm here to help!'

                  Example 2:
                    Agent's input - 'Please check the availability of the Chevrolet Onix 2019 car and inform the customer if the model is available. Then, research in the knowledge base how to resolve the brake problem for the Chevrolet Onix 2019 and provide a detailed response to the customer with the steps for the solution.'
                    Expected response - 'Hello! I've checked that you have a Chevrolet Onix 2019. However, we don't have detailed information about brake issues for this model in our system. Please provide another model!'
                  """

## Functions related to RAG

In [180]:
def establish_connection():
    """
    Connect to Milvus server.
    """
    connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)

def collection_exists() -> bool:
    """
    Check if collection exists in Milvus.
    
    Returns:
        bool: True if the collection exists, otherwise False.
    """
    return utility.has_collection(COLLECTION_NAME)

def collection_query() -> Collection:
    """
    Instantiate a Milvus collection object if it exists.

    Returns:
        Collection or None: The collection object if it exists, otherwise None.
    """
    client = MilvusClient(uri=f"http://{MILVUS_HOST}:19530")

    collection_exist = collection_exists()

    if not collection_exist:
        return None
    else:
        return Collection(COLLECTION_NAME)

def extract_text_from_pdf(file_path: str) -> str:
    """
    Extract text from a PDF file.
    
    Args:
        file_path (str): Path to the PDF file.
        
    Returns:
        str: Extracted text.
    """
    text = ""
    with open(file_path, "rb") as f:
        pdf_reader = PdfReader(f)
        for page in pdf_reader.pages:
            text_content = page.extract_text()
            if text_content:
                text += text_content
    return text

def get_knowledge_base(embeddings) -> Milvus:
    """
    Get the knowledge base collection from Milvus.
    
    Args:
        embeddings: Embedding function for Milvus.
    
    Returns:
        Milvus: Knowledge base collection.
    """
    if collection_exists():
        knowledge_base = Milvus(
            collection_name=COLLECTION_NAME,
            embedding_function=embeddings,
            connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT},
            auto_id=True
        )
        return knowledge_base
    else:
        return None

def process_pdf(pdf_paths, openai_api_key: str) -> Collection:
    """
    Process PDF files and create a knowledge base.
    
    Args:
        pdf_paths (list of str): Paths to the PDF files.
        openai_api_key (str): OpenAI API key.
        
    Returns:
        Collection: Knowledge base collection object.
    """
    if isinstance(pdf_paths, str):
        pdf_paths = [pdf_paths]

    temp_dir = "temp_pdfs"
    os.makedirs(temp_dir, exist_ok=True)

    try:
        for pdf_path in pdf_paths:
            print("Processing PDF:", pdf_path)
            file_path = os.path.join(temp_dir, os.path.basename(pdf_path))
            print("Temporary file path:", file_path)

            try:
                with open(pdf_path, "rb") as f:
                    pdf_data = f.read()
                with open(file_path, "wb") as f:
                    f.write(pdf_data)

            except FileNotFoundError:
                print(f"File not found: {pdf_path}")

            text = extract_text_from_pdf(file_path)
            filename = os.path.basename(file_path)
            brand, model, year = filename.rstrip('.pdf').split('_')

            metadata_text = f"Brand: {brand}, Model: {model}, Year: {year}\n"
            enhanced_text = metadata_text + text

            text_splitter = CharacterTextSplitter(
                separator='\n',
                chunk_size=500,
                chunk_overlap=20,
                length_function=len
            )

            chunks = text_splitter.split_text(enhanced_text)

            embeddings = OpenAIEmbeddings(chunk_size=500, openai_api_key=openai_api_key, model='text-embedding-ada-002')

            metadata = [{'brand': brand, 'model': model, 'year': year} for _ in chunks]

            knowledge_base = get_knowledge_base(embeddings)
            print('knowledge_base', knowledge_base)

            if knowledge_base is not None:
                knowledge_base.add_texts(chunks, metadata, auto_id=True)
            else:
                knowledge_base = Milvus.from_texts(chunks,
                                                   embeddings,
                                                   metadata,
                                                   connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT},
                                                   collection_name=COLLECTION_NAME,
                                                   search_params={"metric_type": "L2", "params": {"nprobe": 10}, "offset": 5})
    
    finally:
        shutil.rmtree(temp_dir)
        print("Temporary files and directory removed.")

    return knowledge_base

In [181]:
def get_stored_cars():
    """
    Retrieve and group stored car information from the Milvus collection.

    Args:
        collection (Collection): Milvus collection object.

    Returns:
        dict: A dictionary grouped by brand, model, and year of available cars.
    """
    expr = "brand != ''"
    fields = ['brand', 'model', 'year']

    collection = collection_query()
    results = collection.query(expr, fields)

    grouped_results = {}

    for result in results:
        brand = result['brand']
        model = result['model']
        year = result['year']

        if brand not in grouped_results:
            grouped_results[brand] = {}

        if model not in grouped_results[brand]:
            grouped_results[brand][model] = set()

        grouped_results[brand][model].add(year)

    return grouped_results


In [ ]:
# Verify the car availability in the knowledge base manually
get_stored_cars()

{'Volkswagen': {'Gol': {'2022'}, 'Taos': {'2023'}}}

In [183]:
@tool
def check_car_availability(brand: str, model: str, year: str) -> str:
    """
    Check if a specific car is available in the knowledge base.
    
    Args:
        brand (str): Brand of the car.
        model (str): Model of the car.
        year (str): Year of the car.
    
    Returns:
        str: "Available" if the car is found, otherwise "Not available".
    """
    knowledge_base = get_knowledge_base(embeddings)
    if not knowledge_base:
        return "Not available"

    stored_cars = get_stored_cars()
    print(f"Stored cars: {stored_cars}")  # Debug print

    if brand in stored_cars and model in stored_cars[brand] and year in stored_cars[brand][model]:
        print(f"Car {brand} {model} {year} is available.")  # Debug print
        return "Available"
    else:
        print(f"Car {brand} {model} {year} is not available.")  # Debug print
        return "Not available"

## Definition of the prompt template

In [184]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template_system),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ],
)

## Establish the connection with the Milvus

In [185]:
establish_connection()

In [186]:
knowledge_base = get_knowledge_base(embeddings=embeddings)
print("Knowledge base exists:", knowledge_base is not None)

Knowledge base exists: True


## Verify the car availability in the knowledge base and add more PDFs if needed

Here you can add more PDFs to the knowledge base if you want. For this, you need to provide the path to the PDF file.

In [187]:
if knowledge_base is None:
    pdf_paths = ["path/to/pdf1.pdf"]
    knowledge_base = process_pdf(pdf_paths, OPENAI_API_KEY)
    print("Knowledge base created.")
else:
    user_input = input("Do you want to add more PDFs to the knowledge base? (y/n): ")
    if user_input.lower() == 'y':
        pdf_path = input("Enter the path of the PDF you want to add: ")
        knowledge_base = process_pdf(pdf_path, OPENAI_API_KEY)
        print("PDF added to the knowledge base.")
    else:
        print("No additional PDFs were added.")

No additional PDFs were added.


## Tools

In [188]:
# Create a retriever tool
retriever = knowledge_base.as_retriever()

In [189]:
retriever_tool = create_retriever_tool(
    retriever,
    "knowledge_base_retriever",
    "Search for information from indexed documents."
)

In [190]:
tools = [check_car_availability, retriever_tool]

## Instantiate the model

In [191]:
llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=OPENAI_API_KEY, max_tokens=500)

## Agent

In [192]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [193]:

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:
result = list(agent_executor.stream(
    {"input": "Please check the availability of the {brand} {model} {year} car in the knowledge base and respond to the customer whether the model is available. Then, research in the knowledge base how to resolve the '{problem_description}' problem for the {brand} {model} {year} and provide a detailed response to the customer with the steps for the solution."}
))

In [ ]:
def extract_responses(results) -> list:
    """
    Extract responses from the agent's results.

    Args:
        results (list): List of results from the agent.
    
    Returns:
        list: List of responses.
    """
    responses = []
    for result in results:
        if 'output' in result:
            responses.append(result['output'])
        else:
            for step in result.get('steps', []):
                if 'observation' in step:
                    responses.append(step['observation'])
                if 'messages' in step:
                    for message in step['messages']:
                        if 'content' in message:
                            responses.append(message['content'])
    return responses

##### 03

In [141]:
result

[{'actions': [ToolAgentAction(tool='check_car_availability', tool_input={'brand': 'Volkswagen', 'model': 'Taos', 'year': '2023'}, log="\nInvoking: `check_car_availability` with `{'brand': 'Volkswagen', 'model': 'Taos', 'year': '2023'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_02OXa8NgfrB7sDTn71LZPulD', 'function': {'arguments': '{"brand": "Volkswagen", "model": "Taos", "year": "2023"}', 'name': 'check_car_availability'}, 'type': 'function'}, {'index': 1, 'id': 'call_Cy0jrJV4kVZ73u5kii94MWd8', 'function': {'arguments': '{"query": "When should the seat heating not be turned on for Volkswagen Taos 2023?"}', 'name': 'knowledge_base_retriever'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-a5f23977-ff08-476d-96fd-e6c91e264de4', tool_calls=[{'name': 'check_car_availability', 'args': {'brand': 'Volkswagen', 'model': 'Taos', 'year': '2023'}, 'id': 'call_02OXa8NgfrB7sDTn71LZPulD'}, {'name': 'k

In [142]:
responses = extract_responses(result)

for response in responses:
    print(response)

Hello! I've checked that you have a Volkswagen Taos 2023, and I'm happy to inform you that this model is available in our knowledge base.

Regarding your question about when the seat heating should not be turned on, here are some important guidelines:

1. **Avoid Using Seat Heating in Certain Conditions**:
   - **If the Seat is Wet**: Do not turn on the seat heating if the seat is wet. This can cause electrical issues and potentially damage the heating elements.
   - **For Infants and Small Children**: Avoid using seat heating for infants and small children as their skin is more sensitive and they may not be able to communicate discomfort.
   - **If You Have Certain Medical Conditions**: Individuals with certain medical conditions, such as those with reduced heat sensitivity, should avoid using seat heating to prevent burns or discomfort.

2. **General Safety Precautions**:
   - **Do Not Leave Unattended**: Never leave the seat heating on when the vehicle is unattended.
   - **Monitor 

##### 02

In [102]:
result

[{'actions': [ToolAgentAction(tool='check_car_availability', tool_input={'brand': 'Volkswagen', 'model': 'Taos', 'year': '2023'}, log="\nInvoking: `check_car_availability` with `{'brand': 'Volkswagen', 'model': 'Taos', 'year': '2023'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_oKsac25ykkyEZMt6zqvUwJkW', 'function': {'arguments': '{"brand": "Volkswagen", "model": "Taos", "year": "2023"}', 'name': 'check_car_availability'}, 'type': 'function'}, {'index': 1, 'id': 'call_etXzETTioETFeq7oQNqBk4RB', 'function': {'arguments': '{"query": "When is the air recirculation mode turned off in Volkswagen Taos 2023?"}', 'name': 'knowledge_base_retriever'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-c7e458ec-9336-4328-aa5f-3fc030c84e8b', tool_calls=[{'name': 'check_car_availability', 'args': {'brand': 'Volkswagen', 'model': 'Taos', 'year': '2023'}, 'id': 'call_oKsac25ykkyEZMt6zqvUwJkW'}, {'name': 'kn

In [103]:
responses = extract_responses(result)

for response in responses:
    print(response)

Hello! I've checked and confirmed that the Volkswagen Taos 2023 is available in our knowledge base. 

Regarding your query about when the air recirculation mode is turned off in the Volkswagen Taos 2023, here are the steps and information you need:

1. **Automatic Deactivation**: The air recirculation mode may automatically turn off in certain conditions to ensure optimal air quality inside the vehicle. This can happen when:
   - The defrost function is activated.
   - The outside temperature is very low, and the system needs to prevent window fogging.
   - The system detects high levels of humidity inside the vehicle.

2. **Manual Deactivation**: You can manually turn off the air recirculation mode by pressing the air recirculation button on the climate control panel. This will allow fresh air from outside to enter the cabin.

3. **System Settings**: Some advanced climate control systems may have settings that allow you to customize when the air recirculation mode is activated or deac

##### 01

In [ ]:
result

[{'actions': [ToolAgentAction(tool='check_car_availability', tool_input={'brand': 'Volkswagen', 'model': 'Taos', 'year': '2023'}, log="\nInvoking: `check_car_availability` with `{'brand': 'Volkswagen', 'model': 'Taos', 'year': '2023'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_wA5TAi8RJObuzkLAAPE4nc3m', 'function': {'arguments': '{"brand": "Volkswagen", "model": "Taos", "year": "2023"}', 'name': 'check_car_availability'}, 'type': 'function'}, {'index': 1, 'id': 'call_f0r9Nuh1WnfRr1V2ZAlc5mPC', 'function': {'arguments': '{"query": "When will the alarm be triggered? Volkswagen Taos 2023"}', 'name': 'knowledge_base_retriever'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-73b73c24-b167-4d69-b0e5-6a379f797c4f', tool_calls=[{'name': 'check_car_availability', 'args': {'brand': 'Volkswagen', 'model': 'Taos', 'year': '2023'}, 'id': 'call_wA5TAi8RJObuzkLAAPE4nc3m'}, {'name': 'knowledge_base_ret

In [ ]:
responses = extract_responses(result)

for response in responses:
    print(response)

Hello! I've checked that you have a Volkswagen Taos 2023, and I'm happy to inform you that this model is available in our system. 

Regarding your question, "When will the alarm be triggered?" for the Volkswagen Taos 2023, here are the details:

The alarm system in your Volkswagen Taos 2023 is designed to be triggered under specific conditions to ensure the security of your vehicle. Here are the steps and conditions under which the alarm will be triggered:

1. **Unauthorized Entry**: The alarm will be triggered if any door, the hood, or the trunk is opened without using the key or remote control.
2. **Ignition Attempt**: If someone tries to start the car without the correct key, the alarm will go off.
3. **Movement Inside the Car**: The interior motion sensors will trigger the alarm if there is movement inside the car when it is locked.
4. **Tilt Sensor**: The alarm will be activated if the car is tilted, which could indicate an attempt to tow the vehicle or steal the wheels.

To ensur